<a href="https://colab.research.google.com/github/comp-aspects-of-appl-linguistics/relation-extraction-utils/blob/master/results/ud_recall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# get stanfordnlp resources 

import os
import stanfordnlp

if not os.path.exists('/root/stanfordnlp_resources'):
  stanfordnlp.download('en', force=True)

In [0]:
# install our utilities

!pip install git+https://github.com/comp-aspects-of-appl-linguistics/relation_extraction_utils.git

In [0]:
# download train.json - whose ID is 1oubGm7qm92HVPARhGLUffiXNkYmC1IkQ (including download of gdrive utility)

!wget -O /tmp/gdrive https://docs.google.com/uc?id=0B3X9GlR6EmbnWksyTEtCM0VfaFE&export=download
!chmod +x /tmp/gdrive
!/tmp/gdrive download 1oubGm7qm92HVPARhGLUffiXNkYmC1IkQ

In [0]:
# get the ud-parsed entries for the relation 'org:founded_by' into a file called org:founded_by_ud.csv

!tac_to_csv --input dev.json --relation org:founded_by > org:founded_by.csv
!parse_ud --input ./org:founded_by.csv --output ./org:founded_by_ud.csv

In [0]:
%%bash

cat > triggers << EOF
create
find
launch
found
s
's
build
her
his
co-founder
start
establish
set
founder
set up
form
EOF

In [0]:
%%bash

cat > paths << "EOF"
!nmod:poss !case >< ^case
^nmod >< ^appos
^obj >< !nsubj
^obj >< ^conj !nsubj
!appos !acl >< !obl
^compound >< ^compound
^obj >< ^acl:relcl ^appos
^obj >< ^acl:relcl
^xcomp >< ^acl
^obj >< ^advcl !nsubj
^nsubj:pass >< !obl
^nmod !appos >< ^appos
^nmod >< !nsubj
!nmod:poss ><
!appos !acl:relcl >< !nsubj
!acl:relcl >< !obl
^obl ^acl:relcl !acl >< !obl
^obj >< ^xcomp !nsubj
^obj >< ^conj ^advcl !nsubj
^obj >< ^acl:relcl !appos
^obj >< ^acl ^obj !nsubj
^nsubj:pass ^acl:relcl >< !acl:relcl !obl
^nsubj:pass !advcl >< !obl
^nsubj:pass >< !obl !nmod !conj !nmod
^nsubj:pass >< !obl !acl !xcomp
^nsubj ^acl:relcl !nmod !nmod:poss ><
^nsubj !obl !nmod !compound >< ^compound
^nsubj !obl >< !nmod
^nsubj !ccomp !ccomp !obl !nmod !appos ><
^nsubj !advcl >< !obl !flat
^nsubj !advcl >< !obl
^nsubj !acl >< !obl
^nsubj >< !obl
^nmod:poss !compound >< ^compound !appos
^nmod:poss >< ^compound
^nmod:poss >< ^appos !appos
^nmod:poss >< ^appos
^nmod ^appos ^nsubj !obj !nmod !compound !punct >< ^punct ^compound ^nmod ^obj !nsubj
^nmod !nmod !acl:relcl >< ^acl:relcl ^nmod ^obl !nsubj
^nmod >< ^acl:relcl
^det !acl:relcl >< !obl
^conj !nmod:poss ><
^conj !conj !nmod:poss ><
^compound ^nmod >< ^conj !appos !nmod
^compound ^nmod >< !nsubj
^compound ^compound ^nmod >< ^appos
^compound ^compound !nmod:poss ><
^compound !nmod:poss !case >< ^case
^appos ^nsubj:pass >< !obl !flat
^appos !nmod:poss !case >< ^case
!nmod:poss !case >< ^case ^nmod:poss
!nmod >< ^nmod ^appos
!appos !acl:relcl >< ^acl:relcl !acl:relcl !nsubj
!appos !acl:relcl >< !obl !nmod
!appos !acl >< !obl !flat
!acl >< !advmod !obl
EOF

In [0]:
!echo "total number of relations:" $(( $(wc -l < org:founded_by_ud.csv) - 1 ))
!echo "total number sentences to consider:" $(( $(grep -v 'produced more than one sentence' org:founded_by_ud.csv |wc -l) - 1 ))
!echo "number of matches:" $(( $(extract_relations_ud --input org:founded_by_ud.csv paths triggers |wc -l) - 1 ))

total number of relations: 76
total number sentences to consider: 72
number of matches: 27


In [0]:
!cat  org:founded_by_ud.csv

tac_tokens,id,sentence,ent1,ent2,ent1_start,ent1_end,ent2_start,ent2_end,ud_parse,words,lemmas,comment
"['The', 'Bipartisan', 'Policy', 'Center', 'is', 'a', 'not-for-profit', 'think', 'tank', 'created', 'in', '2007', 'by', 'four', 'former', 'Senate', 'majority', 'leaders', '--', 'Howard', 'Baker', ',', 'Tom', 'Daschle', ',', 'Bob', 'Dole', 'and', 'George', 'Mitchell', '--', 'to', 'develop', 'possible', 'proposals', 'on', 'complicated', 'issues', 'such', 'as', 'energy', ',', 'transportation', 'and', 'health', 'care', '.']",0,"The Bipartisan Policy Center is a not-for-profit think tank created in 2007 by four former Senate majority leaders - Howard Baker, Tom Daschle, Bob Dole and George Mitchell - to develop possible proposals on complicated issues such as energy, transportation and health care.",Bipartisan Policy Center,Howard Baker,2,4,22,23,"[('1', 'The', 'det', '4', 'Center'), ('2', 'Bipartisan', 'compound', '3', 'Policy'), ('3', 'Policy', 'compound', '4', 'Center'), ('4', 'Center',